In [1]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
def order_book(month,day):
    data1 = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/stat_order_book/CNF14_0'+str(month)+'_'+str(day)+'_order_book_final.csv'
    data1 = pd.read_csv(datapath,sep='\t',encoding = 'utf-8')
    data_book = data1[['0','1','2','3']]
    return data_book

In [4]:
def day_time(month,day):
    data = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.0'+str(month)+'.'+str(day)+'.csv'
    data = pd.read_csv(datapath)
    data_CNG14 = data[data.Series == 'CNG14']
    data = data_CNG14
    market_open_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D09:00')].index.tolist()[0] 
    market_close_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D16:00')].index.tolist()[0] 
    data_open = data[market_open_time:market_close_time + 1]
    timestamp_ = data_open.TimeStamp.unique()
    return timestamp_

In [5]:
def time_transform(timestamp_time):
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600 \
                +float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60\
                +float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second.append(second - 32400.0)
    return  time_second

In [6]:
def bid123_ask123_Q(data_book_28_open):
    Bid1 = []
    Bid2 = []
    Bid3 = []
    Bid1_Quantity = []
    Bid2_Quantity = []
    Bid3_Quantity = []
    Ask1 = []
    Ask2 = []
    Ask3 = []
    Ask1_Quantity = []
    Ask2_Quantity = []
    Ask3_Quantity = []

    TimeStamp = []
    for i in range(1,len(data_book_28_open),4):
        #print data_book_28_open.iloc[i]['0']
        #print data_book_28_open.iloc[i]['2']
        Bid1.append(float(data_book_28_open.iloc[i]['0'])/100.0)
        Bid1_Quantity.append(float(data_book_28_open.iloc[i]['1']))

        Bid2.append(float(data_book_28_open.iloc[i + 1]['0'])/100.0)
        Bid2_Quantity.append(float(data_book_28_open.iloc[i + 1]['1']))  

        Bid3.append(float(data_book_28_open.iloc[i + 2]['0'])/100.0)
        Bid3_Quantity.append(float(data_book_28_open.iloc[i + 2]['1']))  

        Ask1.append(float(data_book_28_open.iloc[i]['2'])/100.0) 
        Ask1_Quantity.append(float(data_book_28_open.iloc[i]['3']))

        Ask2.append(float(data_book_28_open.iloc[i + 1]['2'])/100.0) 
        Ask2_Quantity.append(float(data_book_28_open.iloc[i + 1]['3']))    

        Ask3.append(float(data_book_28_open.iloc[i + 2]['2'])/100.0) 
        Ask3_Quantity.append(float(data_book_28_open.iloc[i + 2]['3']))    
        TimeStamp.append(data_book_28_open.iloc[i-1][1])
    return Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity,    

In [7]:
def rise_ask(Ask1,timestamp_time_second):
    rise_ratio = []
    index = np.where(np.array(timestamp_time_second) >= 600)[0][0]
    for i in range(0,index):
        rise_ratio_ = round((Ask1[i] - Ask1[0])*(1.0)/Ask1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Ask1),1):
        #print timestamp_time_second[:i]
        #print timestamp_time_second[i] - 600
        #print np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        index_start = np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        rise_ratio_ = round((Ask1[i] - Ask1[index_start])*(1.0)/Ask1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return rise_ratio


In [8]:
def rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,close_price,\
             N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,\
             N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
             N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI):
    
    if rise_ratio_ask[index] <= -0.4:
        N_1 = N_1 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            N_1_T = N_1_T + 1
            N_1_E = N_1_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_1_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_1_M = N_1_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_1_TI = N_1_TI + (time_traded - time_start)
            #print N_1_TI
            pass

        else:
            if i < 25200 - 900:
                N_1_E = N_1_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_1_M = N_1_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                #print N_1_M
                N_1_TI = N_1_TI + (900.0)
                #print N_1_TI
            else:
                N_1_E = N_1_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_1_M = N_1_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0) 
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_1_TI = N_1_TI + (time_traded - time_start)
                #print N_1_TI
            pass

    elif -0.4 < rise_ratio_ask[index] <= -0.15:
        N_2 = N_2 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 3
            N_2_T = N_2_T + 1
            N_2_E = N_2_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_2_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_2_M = N_2_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_2_TI = N_2_TI + (time_traded - time_start)
            #print N_2_TI
            pass
        else:
            if i < 25200 - 900:
                N_2_E = N_2_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_2_M = N_2_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_2_TI = N_2_TI + (900.0)
                #print N_2_TI
            else:
                N_2_E = N_2_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_2_M = N_3_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_2_TI = N_2_TI + (time_traded - time_start)
                #print N_2_TI
            pass

    elif -0.15 < rise_ratio_ask[index] <= 0.15:
        N_3 = N_3 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 5
            N_3_T = N_3_T + 1
            N_3_E = N_3_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_3_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_3_M = N_3_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_3_TI = N_3_TI + (time_traded - time_start)
            #print N_3_TI
            pass
        else:
            if i < 25200 - 900:
                N_3_E = N_3_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_3_M = N_3_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_3_TI = N_3_TI + (900.0)
                #print N_3_TI
            else:
                N_3_E = N_3_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_3_M = N_3_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_3_TI = N_3_TI + (time_traded - time_start)
                #print N_3_TI
            pass

    elif 0.15 < rise_ratio_ask[index] < 0.4:
        N_4 = N_4 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 7
            N_4_T = N_4_T + 1
            N_4_E = N_4_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_4_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_4_M = N_4_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_4_TI = N_4_TI + (time_traded - time_start)
            #print N_4_TI
            pass
        else:
            if i < 25200 - 900:
                N_4_E = N_4_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_4_M = N_4_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_4_TI = N_4_TI + (900.0)
                #print N_4_TI
            else:
                N_4_E = N_4_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_4_M = N_4_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_4_TI = N_4_TI + (time_traded - time_start)
                #print N_4_TI
            pass

    elif 0.4 <= rise_ratio_ask[index]:
        N_5 = N_5 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            N_5_T = N_5_T + 1
            N_5_E = N_5_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_5_E      
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_5_M = N_5_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_5_TI = N_5_TI + (time_traded - time_start)
            #print N_5_TI
            pass
        else:
            if i < 25200 - 900:
                N_5_E = N_5_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_5_M = N_5_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_1_TI = N_1_TI + (900.0)
                #print N_5_TI
            else:
                N_5_E = N_5_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_5_M = N_5_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_5_TI = N_5_TI + (time_traded - time_start)
                #print N_5_TI 
            pass
        
    return N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
           N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI


In [9]:
def bid(time1,time2,Bid,Ask,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,\
        Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weigth1,weigth2,weigth3):
    [N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
     N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI] = [0 for i in  range(0,25,1)]
    [N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
     N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI] = [0 for i in  range(0,25,1)]
    [N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
     N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI] = [0 for i in  range(0,25,1)]
    [N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
     N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI] = [0 for i in  range(0,25,1)]
    [N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
     N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI] = [0 for i in  range(0,25,1)]

    for i in range(time1,time2,1):
        #print i
        #print np.where(array(timestamp_time_second) <= i)[0]
        index = np.where(array(timestamp_time_second) <= i)[0][-1]
        #print i
        #print index
        #print timestamp_time_second[index]
        if i < 25200 - 900:
            index_15_min = np.where(np.array(timestamp_time_second[index:]) >= i + 900)[0][0]
            #print index_15_min
            #print timestamp_time_second[index+index_15_min]
        #print 'time stamp = %d'%(timestamp_time_second[index])
        #print index
        #print index + index_15_min 
        #rint Bid1[index]
        #print Ask1[index]
        #print len(Ask1[(index+1):index + index_15_min])

        #if Bid1[index] > min(Ask1[(index+1):index + index_15_min]):
        #    first_min_index = np.where(np.array(Ask1[(index+1):index + index_15_min]) < Bid1[index])[0][0]
        #    print 'mdd = %d'%((max(Ask1[(index + 1):(index + 1 + first_min_index)]) - Bid1[index])*(1.0)/(5.0))
        #print min_Ask1_time_series[index + 1]
        weighted_Bid = Bid1_Quantity[index] * weight1 + Bid2_Quantity[index] * weight2 + Bid3_Quantity[index] * weight3
        weighted_Ask = Ask1_Quantity[index] * weight1 + Ask2_Quantity[index] * weight2 + Ask3_Quantity[index] * weight3  
        #print weighted_Bid 
        #print weighted_Ask
        #print (weighted_Bid*(1.0) / weighted_Ask)

        if (weighted_Bid*(1.0) / weighted_Ask) <= 0.7:
            N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
            N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,\
                                                          N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,\
                                                          N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI)                                               

        elif 0.7 < (weighted_Bid*(1.0) / weighted_Ask) <= 0.9:
            N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
            N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,\
                                                          N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,\
                                                          N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI)                                                

        elif 0.9 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.1:
            N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
            N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,\
                                                          N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,\
                                                          N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI)                                    

        elif 1.1 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.3:
            N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
            N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,\
                                                          N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,\
                                                          N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI)                              

        elif 1.30 < (weighted_Bid*(1.0) / weighted_Ask):
            N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
            N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,\
                                                          N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,\
                                                          N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI)                 

    return [[N_11,N_12,N_13,N_14,N_15],[N_11_T,N_12_T,N_13_T,N_14_T,N_15_T],[N_11_E,N_12_E,N_13_E,N_14_E,N_15_E],\
             [N_11_M,N_12_M,N_13_M,N_14_M,N_15_M],[N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI],\
            [N_21,N_22,N_23,N_24,N_25],[N_21_T,N_22_T,N_23_T,N_24_T,N_25_T],[N_21_E,N_22_E,N_23_E,N_24_E,N_25_E],\
             [N_21_M,N_22_M,N_23_M,N_24_M,N_25_M],[N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI],\
            [N_31,N_32,N_33,N_34,N_35],[N_31_T,N_32_T,N_33_T,N_34_T,N_35_T],[N_31_E,N_32_E,N_33_E,N_34_E,N_35_E],\
             [N_31_M,N_32_M,N_33_M,N_34_M,N_35_M],[N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI],\
            [N_41,N_42,N_43,N_44,N_45],[N_41_T,N_42_T,N_43_T,N_44_T,N_45_T],[N_41_E,N_42_E,N_43_E,N_44_E,N_45_E],\
             [N_41_M,N_42_M,N_43_M,N_44_M,N_45_M],[N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI],\
            [N_51,N_52,N_53,N_54,N_55],[N_51_T,N_52_T,N_53_T,N_54_T,N_55_T],[N_51_E,N_52_E,N_53_E,N_54_E,N_55_E],\
             [N_51_M,N_52_M,N_53_M,N_54_M,N_55_M],[N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI]] 


### day 2/18

In [28]:
data_book = order_book(2,'18')
data_book_open = data_book[1028:301211+1]
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,'18')
timestamp_time_second = time_transform(timestamp_time)

In [12]:
#data_book_open

In [29]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 116.894525


In [30]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 88.897154


In [21]:
#hist(rise_ratio_ask,bins = 150)

9:00~9:15

In [31]:
close_price = 6809
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [32]:
time1 = 0
time2 = 600
mat28_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [33]:
#mat28_1_15,mat28_2_15, mat28_3_15

9:15~11:00

In [34]:
time1 = 600
time2 = 7200
mat28_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

11:00~13:00

In [35]:
time1 = 7200
time2 = 14400
mat28_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

13:00~16:00

In [36]:
time1 = 14400
time2 = 25200
mat28_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/19

In [37]:
data_book = order_book(2,19)
data_book_open = data_book[1152:344127+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,19)
timestamp_time_second = time_transform(timestamp_time)

In [15]:
#data_book_open

In [38]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 117.621838


In [39]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 119.823896


In [40]:
close_price = 6941
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [41]:
time1 = 0
time2 = 600
mat29_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [42]:
time1 = 600
time2 = 7200
mat29_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [43]:
time1 = 7200
time2 = 14400
mat29_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [44]:
time1 = 14400
time2 = 25200
mat29_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/24

In [45]:
data_book = order_book(2,24)
data_book_open = data_book[660:438423+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,24)
timestamp_time_second = time_transform(timestamp_time)

In [18]:
#data_book_open

In [46]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 148.730503


In [47]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 234.808093


In [48]:
close_price = 6625
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [49]:
time1 = 0
time2 = 600
mat27_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [50]:
time1 = 600
time2 = 7200
mat27_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [51]:
time1 = 7200
time2 = 14400
mat27_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [52]:
time1 = 14400
time2 = 25200
mat27_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [64]:
mat27_3_16

[[0, 93, 409, 160, 0],
 [0, 23, 36, 59, 0],
 [0, 36.0, 185.0, 280.0, 0],
 [0, 471.0, 1969.0, 598.0, 0],
 [0, 70951.0, 305347.0, 125567.0, 0],
 [0, 497, 751, 353, 0],
 [0, 114, 54, 79, 0],
 [0, 21.0, 296.0, 450.0, 0],
 [0, 3395.0, 3477.0, 1347.0, 0],
 [0, 321364.0, 621900.0, 293292.0, 0],
 [0, 724, 1194, 515, 0],
 [0, 204, 60, 83, 0],
 [0, 136.0, 157.0, 172.0, 0],
 [0, 5984.0, 5996.0, 2181.0, 0],
 [0, 519666.0, 1040048.0, 417353.0, 0],
 [0, 485, 1743, 505, 0],
 [0, 176, 10, 36, 0],
 [0, 75.0, -957.0, -84.0, 0],
 [0, 2265.0, 8952.0, 2183.0, 0],
 [0, 370543.0, 1491145.0, 434385.0, 0],
 [0, 269, 2581, 521, 0],
 [0, 84, 51, 11, 0],
 [0, -18.0, -2288.0, 203.0, 0],
 [0, 1362.0, 13849.0, 1820.0, 0],
 [0, 216799.0, 2268083.0, 444982.0, 0]]

In [39]:
#pd.DataFrame(array(mat29_1)+array(mat28_1)+array(mat27_1))

In [69]:
a = 0
array(mat[a+1])*1.0/array(mat[a]),array(mat[a+2])/array(mat[a]),array(mat[a+3])/array(mat[a]),array(mat[a+4])/array(mat[a])

(array([ 0.5297619 ,  0.31147541,  0.54868819,  0.88974359,  1.        ]),
 array([-0.88095238, -0.22404372,  0.40665994,  0.96282051,  1.        ]),
 array([ 3.07738095,  3.01639344,  2.01437941,  2.08717949,  1.        ]),
 array([ 450.7202381 ,  657.93989071,  555.90792129,  428.22051282,
         163.85714286]))

### Day 2/25

In [53]:
data_book = order_book(2,25)
data_book_open = data_book[1028:380387+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,25)
timestamp_time_second = time_transform(timestamp_time)

In [54]:
#data_book_open

In [55]:
len(timestamp_time_second)

94840

In [56]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 139.504858


In [57]:
len(Ask1)

94840

In [58]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 322.512873


In [59]:
close_price = 6497
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [60]:
time1 = 0
time2 = 600
mat24_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [61]:
time1 = 600
time2 = 7200
mat24_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [62]:
time1 = 7200
time2 = 14400
mat24_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [63]:
time1 = 14400
time2 = 25200
mat24_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/26

In [65]:
data_book = order_book(2,26)
data_book_open = data_book[892:312443+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,26)
timestamp_time_second = time_transform(timestamp_time)

In [24]:
#data_book_open 

In [66]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 152.622173


In [67]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 220.477081


In [68]:
close_price = 6503
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [69]:
time1 = 0
time2 = 600
mat22_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [70]:
time1 = 600
time2 = 7200
mat22_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [71]:
time1 = 7200
time2 = 14400
mat22_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [72]:
time1 = 14400
time2 = 25200
mat22_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/27

In [73]:
data_book = order_book(2,27)
data_book_open = data_book[444:212171+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,27)
timestamp_time_second = time_transform(timestamp_time)

In [74]:
#data_book_open

In [75]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 76.603320


In [76]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 119.709573


In [77]:
close_price = 6535
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [78]:
time1 = 0
time2 = 600
mat21_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [79]:
time1 = 600
time2 = 7200
mat21_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [80]:
time1 = 7200
time2 = 14400
mat21_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [81]:
time1 = 14400
time2 = 25200
mat21_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/17

In [82]:
data_book = order_book(2,17)
data_book_open = data_book[668:299679+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,17)
timestamp_time_second = time_transform(timestamp_time)

In [59]:
#data_book_open

In [83]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 102.406580


In [84]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 261.770393


In [85]:
close_price = 6936
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [86]:
time1 = 0
time2 = 600
mat20_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [87]:
time1 = 600
time2 = 7200
mat20_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [88]:
time1 = 7200
time2 = 14400
mat20_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [89]:
time1 = 14400
time2 = 25200
mat20_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [90]:
mat20_3_16

[[0, 35, 1420, 91, 0],
 [0, 0, 218, 0, 0],
 [0, -23.0, 666.7999999999913, 68.0, 0],
 [0, 163.0, 5199.0, 336.0, 0],
 [0, 31500.0, 1129939.0, 81900.0, 0],
 [0, 89, 2425, 103, 0],
 [0, 0, 253, 0, 0],
 [0, 9.0, 533.1999999999869, 83.0, 0],
 [0, 400.0, 9165.0, 387.0, 0],
 [0, 80100.0, 2073592.0, 92700.0, 0],
 [0, 92, 3273, 42, 0],
 [0, 0, 91, 0, 0],
 [0, 1.0, 308.4000000000007, 46.0, 0],
 [0, 394.0, 12289.0, 159.0, 0],
 [0, 82800.0, 2909407.0, 37800.0, 0],
 [0, 46, 1346, 26, 0],
 [0, 0, 63, 0, 0],
 [0, -1.0, 17.799999999999937, 27.0, 0],
 [0, 195.0, 5232.0, 100.0, 0],
 [0, 41400.0, 1164367.0, 23400.0, 0],
 [0, 82, 1729, 1, 0],
 [0, 0, 38, 0, 0],
 [0, -26.0, -957.200000000005, 2.0, 0],
 [0, 377.0, 7022.0, 3.0, 0],
 [0, 73800.0, 1327852.0, 900.0, 0]]

In [91]:
SUM_Bid1_15 = \
array(mat20_1_15)+array(mat21_1_15)+array(mat22_1_15)+\
array(mat24_1_15)+array(mat27_1_15)+array(mat28_1_15)+array(mat29_1_15)
SUM_Bid2_15 = \
array(mat20_2_15)+array(mat21_2_15)+array(mat22_2_15)+\
array(mat24_2_15)+array(mat27_2_15)+array(mat28_2_15)+array(mat29_2_15)
SUM_Bid3_15 = \
array(mat20_3_15)+array(mat21_3_15)+array(mat22_3_15)+\
array(mat24_3_15)+array(mat27_3_15)+array(mat28_3_15)+array(mat29_3_15)

In [92]:
SUM_Bid1_11 = \
array(mat20_1_11)+array(mat21_1_11)+array(mat22_1_11)+\
array(mat24_1_11)+array(mat27_1_11)+array(mat28_1_11)+array(mat29_1_11)
SUM_Bid2_11 = \
array(mat20_2_11)+array(mat21_2_11)+array(mat22_2_11)+\
array(mat24_2_11)+array(mat27_2_11)+array(mat28_2_11)+array(mat29_2_11)
SUM_Bid3_11 = \
array(mat20_3_11)+array(mat21_3_11)+array(mat22_3_11)+\
array(mat24_3_11)+array(mat27_3_11)+array(mat28_3_11)+array(mat29_3_11)

In [93]:
SUM_Bid1_13 = \
array(mat20_1_13)+array(mat21_1_13)+array(mat22_1_13)+\
array(mat24_1_13)+array(mat27_1_13)+array(mat28_1_13)+array(mat29_1_13)
SUM_Bid2_13 = \
array(mat20_2_13)+array(mat21_2_13)+array(mat22_2_13)+\
array(mat24_2_13)+array(mat27_2_13)+array(mat28_2_13)+array(mat29_2_13)
SUM_Bid3_13 = \
array(mat20_3_13)+array(mat21_3_13)+array(mat22_3_13)+\
array(mat24_3_13)+array(mat27_3_13)+array(mat28_3_13)+array(mat29_3_13)

In [94]:
SUM_Bid1_16 = \
array(mat20_1_16)+array(mat21_1_16)+array(mat22_1_16)+\
array(mat24_1_16)+array(mat27_1_16)+array(mat28_1_16)+array(mat29_1_16)
SUM_Bid2_16 = \
array(mat20_2_16)+array(mat21_2_16)+array(mat22_2_16)+\
array(mat24_2_16)+array(mat27_2_16)+array(mat28_2_16)+array(mat29_2_16)
SUM_Bid3_16 = \
array(mat20_3_16)+array(mat21_3_16)+array(mat22_3_16)+\
array(mat24_3_16)+array(mat27_3_16)+array(mat28_3_16)+array(mat29_3_16)

In [95]:
SUM_Bid1_15

array([[  0.00000000e+00,   4.87000000e+02,   1.33300000e+03,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.62000000e+02,   3.09000000e+02,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,  -9.49000000e+02,  -3.17300000e+03,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   2.19100000e+03,   7.86100000e+03,
          4.50000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.01083000e+05,   1.00542300e+06,
          1.09483000e+05,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   3.85000000e+02,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   1.89000000e+02,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   7.00000000e+00,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   2.50000000e+01,   1.59300000e+03,
          3.48000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.0265000

In [96]:
SUM_Bid2_15

array([[  0.00000000e+00,   4.87000000e+02,   1.33300000e+03,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.62000000e+02,   2.34000000e+02,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,  -5.87000000e+02,  -2.94900000e+03,
          4.05000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   2.67800000e+03,   9.22600000e+03,
          6.57000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.03814000e+05,   1.12332100e+06,
          1.09483000e+05,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   3.85000000e+02,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   1.46000000e+02,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.20000000e+01,   1.13000000e+02,
          2.40000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   4.10000000e+01,   2.01800000e+03,
          4.68000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.0393000

In [97]:
SUM_Bid3_15

array([[  0.00000000e+00,   4.87000000e+02,   1.33300000e+03,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.62000000e+02,   2.19000000e+02,
          1.98000000e+02,   0.00000000e+00],
       [  0.00000000e+00,  -2.25000000e+02,  -2.72200000e+03,
          6.03000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.16500000e+03,   1.05590000e+04,
          8.55000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   3.04801000e+05,   1.14758200e+06,
          1.09618000e+05,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   3.85000000e+02,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.60000000e+01,   1.46000000e+02,
          1.20000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   4.80000000e+01,   2.59000000e+02,
          3.60000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   5.70000000e+01,   2.40300000e+03,
          5.88000000e+02,   0.00000000e+00],
       [  0.00000000e+00,   1.0441000

In [98]:
SUM_Bid1_11

array([[  3.91000000e+02,   1.27600000e+03,   4.21000000e+03,
          7.67000000e+02,   4.30000000e+02],
       [  3.59000000e+02,   1.06800000e+03,   3.24200000e+03,
          6.91000000e+02,   4.02000000e+02],
       [  2.65000000e+02,   6.80000000e+02,   2.03400000e+03,
          4.13000000e+02,   4.14000000e+02],
       [  8.09000000e+02,   2.62800000e+03,   9.22300000e+03,
          1.45400000e+03,   1.13400000e+03],
       [  1.08169000e+05,   4.00851000e+05,   1.55153400e+06,
          1.87000000e+05,   1.40225000e+05],
       [  5.42000000e+02,   2.01100000e+03,   5.00500000e+03,
          1.04700000e+03,   4.27000000e+02],
       [  4.46000000e+02,   1.50700000e+03,   3.57700000e+03,
          8.80000000e+02,   4.07000000e+02],
       [  2.26000000e+02,   4.13000000e+02,   1.35500000e+03,
          6.95000000e+02,   4.53000000e+02],
       [  1.44600000e+03,   5.15500000e+03,   1.28910000e+04,
          2.59400000e+03,   1.07900000e+03],
       [  1.69801000e+05,   8.1676900

In [99]:
SUM_Bid2_11

array([[  3.91000000e+02,   1.27600000e+03,   4.21000000e+03,
          7.67000000e+02,   4.30000000e+02],
       [  2.99000000e+02,   8.32000000e+02,   2.49500000e+03,
          5.54000000e+02,   3.91000000e+02],
       [  4.52000000e+02,   1.17600000e+03,   3.71700000e+03,
          7.24000000e+02,   7.82000000e+02],
       [  1.33300000e+03,   4.22900000e+03,   1.44250000e+04,
          2.58300000e+03,   1.61300000e+03],
       [  1.94396000e+05,   6.46261000e+05,   2.35856000e+06,
          3.51816000e+05,   1.98347000e+05],
       [  5.42000000e+02,   2.01100000e+03,   5.00500000e+03,
          1.04700000e+03,   4.27000000e+02],
       [  3.74000000e+02,   1.15500000e+03,   2.56200000e+03,
          6.34000000e+02,   3.69000000e+02],
       [  4.35000000e+02,   1.18900000e+03,   2.93700000e+03,
          1.17500000e+03,   7.62000000e+02],
       [  2.16100000e+03,   7.58000000e+03,   1.91310000e+04,
          3.83600000e+03,   1.61000000e+03],
       [  3.03546000e+05,   1.1896960

In [100]:
SUM_Bid3_11

array([[  3.91000000e+02,   1.27600000e+03,   4.21000000e+03,
          7.67000000e+02,   4.30000000e+02],
       [  2.07000000e+02,   6.43000000e+02,   1.51800000e+03,
          4.02000000e+02,   3.23000000e+02],
       [  4.95000000e+02,   1.53100000e+03,   4.92000000e+03,
          1.13400000e+03,   1.11400000e+03],
       [  1.77000000e+03,   5.75900000e+03,   1.90200000e+04,
          3.39100000e+03,   2.09600000e+03],
       [  3.26984000e+05,   8.42357000e+05,   2.97719200e+06,
          4.81096000e+05,   1.90889000e+05],
       [  5.42000000e+02,   2.01100000e+03,   5.00500000e+03,
          1.04700000e+03,   4.27000000e+02],
       [  2.68000000e+02,   9.69000000e+02,   1.64200000e+03,
          4.09000000e+02,   3.34000000e+02],
       [  4.39000000e+02,   1.95900000e+03,   4.24300000e+03,
          1.45200000e+03,   1.09700000e+03],
       [  2.82800000e+03,   9.72600000e+03,   2.43680000e+04,
          4.95700000e+03,   2.05900000e+03],
       [  4.32420000e+05,   1.4071440

In [101]:
SUM_Bid1_13

array([[  1.20000000e+01,   1.43000000e+02,   1.19020000e+04,
          1.23400000e+03,   4.00000000e+00],
       [  0.00000000e+00,   4.10000000e+01,   5.04000000e+02,
          2.59000000e+02,   2.00000000e+00],
       [ -3.40000000e+01,  -1.40000000e+02,  -1.38400000e+03,
          3.51000000e+02,   4.00000000e+00],
       [  6.00000000e+01,   4.64000000e+02,   1.82460000e+04,
          2.35700000e+03,   4.00000000e+00],
       [  1.26000000e+04,   9.61780000e+04,   1.04114430e+07,
          9.34191000e+05,   1.82000000e+02],
       [  1.90000000e+01,   3.24000000e+02,   2.82400000e+03,
          1.12800000e+03,   5.00000000e+00],
       [  0.00000000e+00,   6.60000000e+01,   6.81000000e+02,
          2.12000000e+02,   5.00000000e+00],
       [ -7.00000000e+01,  -2.45000000e+02,  -1.37100000e+03,
         -3.30000000e+01,   1.00000000e+01],
       [  1.03000000e+02,   9.98000000e+02,   6.64800000e+03,
          2.85600000e+03,   1.00000000e+01],
       [  1.71000000e+04,   2.3653300

In [102]:
SUM_Bid2_13

array([[  1.20000000e+01,   1.43000000e+02,   1.19020000e+04,
          1.23400000e+03,   4.00000000e+00],
       [  0.00000000e+00,   4.00000000e+00,   4.50000000e+01,
          6.50000000e+01,   2.00000000e+00],
       [ -3.40000000e+01,  -2.10000000e+02,  -1.77700000e+03,
          4.83000000e+02,   6.00000000e+00],
       [  7.20000000e+01,   6.97000000e+02,   3.09050000e+04,
          3.66700000e+03,   8.00000000e+00],
       [  1.26000000e+04,   1.25157000e+05,   1.06782870e+07,
          1.06681700e+06,   3.76000000e+02],
       [  1.90000000e+01,   3.24000000e+02,   2.82400000e+03,
          1.12800000e+03,   5.00000000e+00],
       [  0.00000000e+00,   4.20000000e+01,   1.32000000e+02,
          1.01000000e+02,   0.00000000e+00],
       [ -7.00000000e+01,  -2.76000000e+02,  -2.01900000e+03,
         -1.38000000e+02,  -1.00000000e+01],
       [  1.22000000e+02,   1.38500000e+03,   1.07980000e+04,
          4.17100000e+03,   3.00000000e+01],
       [  2.16000000e+04,   2.5634000

In [103]:
SUM_Bid3_13

array([[  1.20000000e+01,   1.43000000e+02,   1.19020000e+04,
          1.23400000e+03,   4.00000000e+00],
       [  0.00000000e+00,   4.00000000e+00,   3.50000000e+01,
          4.00000000e+01,   0.00000000e+00],
       [ -3.40000000e+01,  -2.06000000e+02,  -1.76300000e+03,
          4.53000000e+02,  -2.00000000e+00],
       [  8.40000000e+01,   8.40000000e+02,   4.28100000e+04,
          4.94100000e+03,   1.80000000e+01],
       [  1.44000000e+04,   1.25441000e+05,   1.06883630e+07,
          1.08756000e+06,   0.00000000e+00],
       [  1.90000000e+01,   3.24000000e+02,   2.82400000e+03,
          1.12800000e+03,   5.00000000e+00],
       [  0.00000000e+00,   4.20000000e+01,   8.60000000e+01,
          6.00000000e+01,   0.00000000e+00],
       [ -7.00000000e+01,  -2.34000000e+02,  -2.06200000e+03,
         -1.24000000e+02,  -1.00000000e+01],
       [  1.41000000e+02,   1.70900000e+03,   1.36400000e+04,
          5.33300000e+03,   3.50000000e+01],
       [  2.16000000e+04,   2.5932200

In [104]:
SUM_Bid1_16

array([[  4.80000000e+01,   1.47500000e+03,   7.82900000e+03,
          1.42800000e+03,   3.83000000e+02],
       [  8.00000000e+00,   8.26000000e+02,   4.25900000e+03,
          9.23000000e+02,   2.32000000e+02],
       [ -2.97000000e+02,  -1.00000000e+01,   1.89360000e+03,
          2.88000000e+02,  -3.70000000e+02],
       [  3.96000000e+02,   3.85500000e+03,   1.28260000e+04,
          2.98400000e+03,   1.42900000e+03],
       [  1.72279000e+05,   7.72261000e+05,   4.15611900e+06,
          6.98865000e+05,   5.03330000e+04],
       [  5.60000000e+01,   2.22800000e+03,   1.08230000e+04,
          1.63400000e+03,   3.99000000e+02],
       [  1.20000000e+01,   1.29000000e+03,   5.53500000e+03,
          7.97000000e+02,   1.43000000e+02],
       [ -3.02000000e+02,   5.50000000e+02,   6.49200000e+02,
         -5.22000000e+02,  -8.87000000e+02],
       [  4.28000000e+02,   6.05100000e+03,   2.20300000e+04,
          4.61900000e+03,   2.57300000e+03],
       [  2.70604000e+05,   1.0984390

In [105]:
SUM_Bid2_16

array([[  4.80000000e+01,   1.47500000e+03,   7.82900000e+03,
          1.42800000e+03,   3.83000000e+02],
       [  8.00000000e+00,   6.31000000e+02,   2.53300000e+03,
          5.65000000e+02,   6.60000000e+01],
       [ -2.89000000e+02,   4.22000000e+02,   3.21420000e+03,
          5.10000000e+02,  -3.64000000e+02],
       [  4.47000000e+02,   5.58800000e+03,   2.22560000e+04,
          4.63800000e+03,   1.92500000e+03],
       [  3.22224000e+05,   9.61252000e+05,   5.53539600e+06,
          1.00015700e+06,   1.79200000e+04],
       [  5.60000000e+01,   2.22800000e+03,   1.08230000e+04,
          1.63400000e+03,   3.99000000e+02],
       [  1.20000000e+01,   8.54000000e+02,   2.42000000e+03,
          4.37000000e+02,   1.08000000e+02],
       [ -2.90000000e+02,   1.08400000e+03,   6.50800000e+02,
         -2.08000000e+02,  -7.93000000e+02],
       [  4.86000000e+02,   8.87500000e+03,   3.37730000e+04,
          6.32800000e+03,   2.99000000e+03],
       [  3.02411000e+05,   1.4454800

In [106]:
a = 0
Bid1_15_1 = pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

In [107]:
a = 5
Bid1_15_2 = pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

In [108]:
a = 10
Bid1_15_3 = pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

In [109]:
a = 15
Bid1_15_4 = pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

In [110]:
a = 20
Bid1_15_5 = pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

In [111]:
pd.concat([Bid1_15_1,Bid1_15_2,Bid1_15_3,Bid1_15_4,Bid1_15_5])

,0,1,2,3,4
0,0.0,487.000000,1333.000000,198.000000,0.0
1,NaN,0.743326,0.231808,1.000000,NaN
2,NaN,-1.948665,-2.380345,1.000000,NaN
3,NaN,4.498973,5.897224,2.272727,NaN
4,NaN,618.240246,754.255814,552.944444,NaN
0,0.0,16.000000,385.000000,120.000000,0.0
1,NaN,1.000000,0.490909,1.000000,NaN
2,NaN,1.000000,0.018182,1.000000,NaN
3,NaN,1.562500,4.137662,2.900000,NaN
4,NaN,641.562500,754.181818,576.808333,NaN


In [112]:
pd.concat([Bid1_15_1,Bid1_15_2,Bid1_15_3,Bid1_15_4,Bid1_15_5]).to_csv('bid1_15_218_227.csv')

In [113]:
a = 0
Bid2_15_1 = pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

In [114]:
a = 5
Bid2_15_2 = pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

In [115]:
a = 10
Bid2_15_3 = pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

In [116]:
a = 15
Bid2_15_4 = pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

In [117]:
a = 20
Bid2_15_5 = pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

In [118]:
pd.concat([Bid2_15_1,Bid2_15_2,Bid2_15_3,Bid2_15_4,Bid2_15_5])

,0,1,2,3,4
0,0.0,487.000000,1333.000000,198.000000,0.0
1,NaN,0.743326,0.175544,1.000000,NaN
2,NaN,-1.205339,-2.212303,2.045455,NaN
3,NaN,5.498973,6.921230,3.318182,NaN
4,NaN,623.848049,842.701425,552.944444,NaN
0,0.0,16.000000,385.000000,120.000000,0.0
1,NaN,1.000000,0.379221,1.000000,NaN
2,NaN,2.000000,0.293506,2.000000,NaN
3,NaN,2.562500,5.241558,3.900000,NaN
4,NaN,649.562500,863.340260,576.808333,NaN


In [120]:
pd.concat([Bid2_15_1,Bid2_15_2,Bid2_15_3,Bid2_15_4,Bid2_15_5]).to_csv('bid2_15_218_227.csv')

In [121]:
a = 0
Bid3_15_1 = pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

In [122]:
a = 5
Bid3_15_2 = pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

In [123]:
a = 10
Bid3_15_3 = pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

In [124]:
a = 15
Bid3_15_4 = pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

In [125]:
a = 20
Bid3_15_5 = pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

In [126]:
pd.concat([Bid3_15_1,Bid3_15_2,Bid3_15_3,Bid3_15_4,Bid3_15_5])

,0,1,2,3,4
0,0.0,487.000000,1333.000000,198.000000,0.0
1,NaN,0.743326,0.164291,1.000000,NaN
2,NaN,-0.462012,-2.042011,3.045455,NaN
3,NaN,6.498973,7.921230,4.318182,NaN
4,NaN,625.874743,860.901725,553.626263,NaN
0,0.0,16.000000,385.000000,120.000000,0.0
1,NaN,1.000000,0.379221,1.000000,NaN
2,NaN,3.000000,0.672727,3.000000,NaN
3,NaN,3.562500,6.241558,4.900000,NaN
4,NaN,652.562500,867.441558,576.808333,NaN


In [127]:
pd.concat([Bid3_15_1,Bid3_15_2,Bid3_15_3,Bid3_15_4,Bid3_15_5]).to_csv('bid3_15_218_227.csv')

In [128]:
a = 0
Bid1_11_1 = pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

In [129]:
a = 5
Bid1_11_2 = pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

In [130]:
a = 10
Bid1_11_3 = pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

In [131]:
a = 15
Bid1_11_4 = pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

In [132]:
a = 20
Bid1_11_5 = pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

In [133]:
pd.concat([Bid1_11_1,Bid1_11_2,Bid1_11_3,Bid1_11_4,Bid1_11_5])

,0,1,2,3,4
0,391.000000,1276.000000,4210.000000,767.000000,430.000000
1,0.918159,0.836991,0.770071,0.900913,0.934884
2,0.677749,0.532915,0.483135,0.538462,0.962791
3,2.069054,2.059561,2.190736,1.895698,2.637209
4,276.647059,314.146552,368.535392,243.807040,326.104651
0,542.000000,2011.000000,5005.000000,1047.000000,427.000000
1,0.822878,0.749378,0.714685,0.840497,0.953162
2,0.416974,0.205370,0.270729,0.663801,1.060890
3,2.667897,2.563401,2.575624,2.477555,2.526932
4,313.285978,406.150671,436.222777,366.024833,321.494145


In [134]:
pd.concat([Bid1_11_1,Bid1_11_2,Bid1_11_3,Bid1_11_4,Bid1_11_5]).to_csv('bid1_11_218_227.csv')

In [135]:
a = 0
Bid2_11_1 = pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

In [136]:
a = 5
Bid2_11_2 = pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

In [137]:
a = 10
Bid2_11_3 = pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

In [138]:
a = 15
Bid2_11_4 = pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

In [139]:
a = 20
Bid2_11_5 = pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

In [140]:
pd.concat([Bid2_11_1,Bid2_11_2,Bid2_11_3,Bid2_11_4,Bid2_11_5])

,0,1,2,3,4
0,391.000000,1276.000000,4210.000000,767.000000,430.000000
1,0.764706,0.652038,0.592637,0.722295,0.909302
2,1.156010,0.921630,0.882898,0.943937,1.818605
3,3.409207,3.314263,3.426366,3.367666,3.751163
4,497.176471,506.474138,560.228029,458.691004,461.272093
0,542.000000,2011.000000,5005.000000,1047.000000,427.000000
1,0.690037,0.574341,0.511888,0.605540,0.864169
2,0.802583,0.591248,0.586813,1.122254,1.784543
3,3.987085,3.769269,3.822378,3.663801,3.770492
4,560.047970,591.594232,635.405994,551.232092,404.730679


In [141]:
pd.concat([Bid2_11_1,Bid2_11_2,Bid2_11_3,Bid2_11_4,Bid2_11_5]).to_csv('bid2_11_218_227.csv')

In [142]:
a = 0
Bid3_11_1 = pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

In [143]:
a = 5
Bid3_11_2 = pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

In [144]:
a = 10
Bid3_11_3 = pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

In [145]:
a = 15
Bid3_11_4 = pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

In [146]:
a = 20
Bid3_11_5 = pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

In [147]:
pd.concat([Bid3_11_1,Bid3_11_2,Bid3_11_3,Bid3_11_4,Bid3_11_5])

,0,1,2,3,4
0,391.000000,1276.000000,4210.000000,767.000000,430.000000
1,0.529412,0.503918,0.360570,0.524120,0.751163
2,1.265985,1.199843,1.168646,1.478488,2.590698
3,4.526854,4.513323,4.517815,4.421121,4.874419
4,836.276215,660.154389,707.171496,627.243807,443.927907
0,542.000000,2011.000000,5005.000000,1047.000000,427.000000
1,0.494465,0.481850,0.328072,0.390640,0.782201
2,0.809963,0.974142,0.847752,1.386819,2.569087
3,5.217712,4.836400,4.868731,4.734479,4.822014
4,797.822878,699.723521,749.986214,706.257880,413.526932


In [148]:
pd.concat([Bid3_11_1,Bid3_11_2,Bid3_11_3,Bid3_11_4,Bid3_11_5]).to_csv('bid3_11_218_227.csv')

In [149]:
a = 0
Bid1_13_1 = pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

In [150]:
a = 5
Bid1_13_2 = pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

In [151]:
a = 10
Bid1_13_3 = pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

In [152]:
a = 15
Bid1_13_4 = pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

In [153]:
a = 20
Bid1_13_5 = pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

In [154]:
pd.concat([Bid1_13_1,Bid1_13_2,Bid1_13_3,Bid1_13_4,Bid1_13_5])

,0,1,2,3,4
0,12.000000,143.000000,11902.000000,1234.000000,4.0
1,0.000000,0.286713,0.042346,0.209887,0.5
2,-2.833333,-0.979021,-0.116283,0.284441,1.0
3,5.000000,3.244755,1.533020,1.910049,1.0
4,1050.000000,672.573427,874.764157,757.042950,45.5
0,19.000000,324.000000,2824.000000,1128.000000,5.0
1,0.000000,0.203704,0.241147,0.187943,1.0
2,-3.684211,-0.756173,-0.485482,-0.029255,2.0
3,5.421053,3.080247,2.354108,2.531915,2.0
4,900.000000,730.040123,738.808782,763.507979,137.4


In [155]:
pd.concat([Bid1_13_1,Bid1_13_2,Bid1_13_3,Bid1_13_4,Bid1_13_5]).to_csv('bid1_13_218_227.csv')

In [156]:
a = 0
Bid2_13_1 = pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

In [157]:
a = 5
Bid2_13_2 = pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

In [158]:
a = 10
Bid2_13_3 = pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

In [159]:
a = 15
Bid2_13_4 = pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

In [160]:
a = 20
Bid2_13_5 = pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

In [161]:
pd.concat([Bid2_13_1,Bid2_13_2,Bid2_13_3,Bid2_13_4,Bid2_13_5])

,0,1,2,3,4
0,12.000000,143.000000,11902.000000,1234.000000,4.0
1,0.000000,0.027972,0.003781,0.052674,0.5
2,-2.833333,-1.468531,-0.149303,0.391410,1.5
3,6.000000,4.874126,2.596622,2.971637,2.0
4,1050.000000,875.223776,897.184255,864.519449,94.0
0,19.000000,324.000000,2824.000000,1128.000000,5.0
1,0.000000,0.129630,0.046742,0.089539,0.0
2,-3.684211,-0.851852,-0.714943,-0.122340,-2.0
3,6.421053,4.274691,3.823654,3.697695,6.0
4,1136.842105,791.172840,867.194405,836.958333,0.0


In [162]:
pd.concat([Bid2_13_1,Bid2_13_2,Bid2_13_3,Bid2_13_4,Bid2_13_5]).to_csv('bid2_13_218_227.csv')

In [219]:
a = 0
Bid3_13_1 = pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

In [220]:
a = 5
Bid3_13_2 = pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

In [221]:
a = 10
Bid3_13_3 = pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

In [222]:
a = 15
Bid3_13_4 = pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

In [223]:
a = 20
Bid3_13_5 = pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

In [224]:
pd.concat([Bid3_13_1,Bid3_13_2,Bid3_13_3,Bid3_13_4,Bid3_13_5])

,0,1,2,3,4
0,18.000000,80.000000,15700.000000,278.000000,22.000000
1,0.000000,0.000000,0.002102,0.057554,0.000000
2,-4.944444,-0.900000,0.069299,-0.794964,-0.136364
3,8.833333,4.525000,3.338790,4.913669,5.000000
4,2000.000000,900.000000,898.928917,896.949640,0.000000
0,41.000000,167.000000,3870.000000,390.000000,21.000000
1,0.000000,0.000000,0.004651,0.043590,0.000000
2,-5.097561,-1.856287,-0.904134,-0.833333,-0.285714
3,8.853659,5.508982,4.488889,5.169231,5.000000
4,1360.975610,900.000000,897.739535,871.964103,0.000000


In [225]:
pd.concat([Bid3_13_1,Bid3_13_2,Bid3_13_3,Bid3_13_4,Bid3_13_5]).to_csv('bid3_13_218_227.csv')

In [226]:
a = 0
Bid1_16_1 = pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

In [227]:
a = 5
Bid1_16_2 = pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

In [228]:
a = 10
Bid1_16_3 = pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

In [229]:
a = 15
Bid1_16_4 = pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

In [230]:
a = 20
Bid1_16_5 = pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

In [231]:
pd.concat([Bid1_16_1,Bid1_16_2,Bid1_16_3,Bid1_16_4,Bid1_16_5])

,0,1,2,3,4
0,19.000000,579.000000,15718.000000,967.000000,130.000000
1,0.000000,0.549223,0.425309,0.502585,0.800000
2,0.947368,0.258031,-0.863659,0.287487,0.792308
3,1.052632,5.849741,1.998473,2.356774,3.115385
4,2131.578947,449.625216,584.395025,581.586350,145.223077
0,67.000000,720.000000,13521.000000,847.000000,79.000000
1,0.000000,0.422222,0.429184,0.453365,0.683544
2,0.970149,0.107222,-0.067007,-0.298701,0.645570
3,1.029851,7.362500,2.112788,3.238489,4.291139
4,1235.820896,583.183333,668.164337,624.229044,185.455696


In [233]:
pd.concat([Bid1_16_1,Bid1_16_2,Bid1_16_3,Bid1_16_4,Bid1_16_5]).to_csv('bid1_16_218_227.csv')

In [234]:
a = 0
Bid2_16_1 = pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

In [235]:
a = 5
Bid2_16_2 = pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

In [236]:
a = 10
Bid2_16_3 = pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

In [237]:
a = 15
Bid2_16_4 = pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

In [238]:
a = 20
Bid2_16_5 = pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

In [239]:
pd.concat([Bid2_16_1,Bid2_16_2,Bid2_16_3,Bid2_16_4,Bid2_16_5])

,0,1,2,3,4
0,19.000000,579.000000,15718.000000,967.000000,130.000000
1,0.000000,0.174439,0.181448,0.283351,0.746154
2,0.947368,0.510190,-0.960224,0.454395,1.592308
3,2.052632,9.713299,3.064067,3.435367,4.115385
4,2463.157895,586.949914,725.552424,760.786970,319.361538
0,67.000000,720.000000,13521.000000,847.000000,79.000000
1,0.000000,0.370833,0.176540,0.244392,0.544304
2,0.970149,0.450278,-0.028282,-0.132704,1.329114
3,2.029851,11.145833,3.156793,4.330579,5.291139
4,1383.582090,688.226389,803.807928,754.744982,185.215190


In [240]:
pd.concat([Bid2_16_1,Bid2_16_2,Bid2_16_3,Bid2_16_4,Bid2_16_5]).to_csv('bid2_16_218_227.csv')

In [241]:
a = 0
Bid3_16_1 = pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

In [242]:
a = 5
Bid3_16_2 = pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

In [243]:
a = 10
Bid3_16_3 = pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

In [244]:
a = 15
Bid3_16_4 = pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

In [245]:
a = 20
Bid3_16_5 = pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

In [246]:
pd.concat([Bid3_16_1,Bid3_16_2,Bid3_16_3,Bid3_16_4,Bid3_16_5])

,0,1,2,3,4
0,19.000000,579.000000,15718.000000,967.000000,130.000000
1,0.000000,0.150259,0.085507,0.189245,0.569231
2,0.947368,0.660449,-0.907495,0.615719,2.338462
3,3.052632,13.466321,4.064639,4.435367,5.115385
4,3552.631579,640.008636,775.171141,830.152017,344.830769
0,67.000000,720.000000,13521.000000,847.000000,79.000000
1,0.000000,0.316667,0.094446,0.186541,0.430380
2,0.970149,0.760000,0.049848,0.056198,1.873418
3,3.029851,14.812500,4.159086,5.330579,6.291139
4,1504.477612,793.095833,844.908513,814.269185,231.708861


In [247]:
pd.concat([Bid3_16_1,Bid3_16_2,Bid3_16_3,Bid3_16_4,Bid3_16_5]).to_csv('bid3_16_218_227.csv')